# Phase 2

### Imports

In [1]:
from pyspark.sql import SparkSession

### Reading Data

In [6]:
spark = SparkSession.builder.getOrCreate()

hotel_booking = spark.read.csv(
    "../phase1_output_datasets/hotel_booking_cleaned.csv",
    header=True,
    inferSchema=True,
)
customer_reservations = spark.read.csv(
    "../phase1_output_datasets/customer_reservations.csv",
    header=True,
    inferSchema=True,
)
all_bookings = spark.read.csv(
    "../phase1_output_datasets/all_bookings.csv",
    header=True,
    inferSchema=True,
)

## 2.1 Spark Analysis

### Cancellation rates
Calculate cancellation rates for each month

### Averages
Compute average price and average number of nights for each month

### Monthly Bookings
Count monthly bookings by market segment. In categories, the term TA means Travel Agents and TO means Tour Operators

### Seasonality
Identify the most popular month of the year for bookings based on revenue